This code develops an application which intelligently processes email order requests and customer inquiries for a fashion store. The system should categorize emails into product inquiries or order requests and generate appropriate responses based on product catalogue information and stock status. 

Table 'Solving_Business_Problems_with_AI.xlsx' gives an example of the product information and customer message. The results are outputed into table 'email_results.xlsx'. 

To make the full use of this code, you need to have enough tokens to gain access to the 'gpt-4o' model using OpenAI. The code primarily uses the gpt-4o model, pandas library and fuzzy matching. 

In [ ]:
# Install the packages.
%pip install openai
%pip install pandas
%pip install xlsxwriter
%pip install openpyxl
%pip install fuzzywuzzy
%pip install python-Levenshtein

CLASSIFY EMAILS 

Read the message (customers's emails) from 'email' sheet in table 'Solving_Business_Problems_with_AI.xlsx'. Each message is classified as either a "product inquiry" or an "order request". 

The results are outputed in 'email_classification' sheet in table 'email_results.xlsx'. 

In [1]:
import pandas as pd
from openai import OpenAI

products_df = pd.read_excel('/Users/jitongd/Desktop/crossover/Solving_Business_Problems_with_AI.xlsx', sheet_name='products')
emails_df = pd.read_excel('/Users/jitongd/Desktop/crossover/Solving_Business_Problems_with_AI.xlsx', sheet_name='emails')

client = OpenAI(
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    api_key='<replace_with_your_own_key>'
)


def classify_email(email_body):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": f"Classify the following email as either 'product inquiry' or 'order request':\n\n{email_body}"}
        ]
    )

    # Correctly access the message content from the response
    message_content = completion.choices[0].message.content.strip().lower()
    
    # Determine the category
    if 'inquiry' in message_content:
        return 'product inquiry'
    elif 'order' in message_content:
        return 'order request'
    else:
        return 'unknown'


# Classify emails
emails_df['category'] = emails_df['message'].apply(classify_email)


output_file_path = 'email_results.xlsx'
classification_results = emails_df[['email_id', 'category']]
classification_results.to_excel(output_file_path, sheet_name='email-classification', index=False)

PROCESS ORDER REQUESTS

For those messages identified as order requests, they go through further processes including identifying product id, quantity, stock checking and updating. The product information is given in 'products' sheet in table 'Solving_Business_Problems_with_AI.xlsx'.

The main steps are summarised below:
1) identify the product name and id
2) check the stock availability
3) figure out the quantity available to the customer according to the stock availability
4) update the status: if stock available, the status is 'created' and update the stock availability; if stock unavailable, the status is 'out-of-stock'

Room for improvement for this code: 
1) capture multiple products requested by one customer

The results are outputed in 'order_status' sheet in table 'email_results.xlsx'. 

In [2]:
from openai import OpenAI
import pandas as pd
from fuzzywuzzy import process

# Load products data
products_df = pd.read_excel('/Users/jitongd/Desktop/crossover/Solving_Business_Problems_with_AI.xlsx', sheet_name='products')

client = OpenAI(
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    api_key='<replace_with_your_own_key>'
)

def extract_product_info_from_message(message):
    prompt = (
        "Extract the product name and quantity from the following message:\n\n"
        f"{message}\n\n"
        "The output should be in the format: Product Name: [product_name], Quantity: [quantity]."
    )

    # Call the OpenAI API to get the response
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    # Correctly access the message content from the response
    response_content = completion.choices[0].message.content.strip()
    return response_content

def parse_product_info(response_content):
    try:
        # Example response format: "Product Name: XYZ, Quantity: 5"
        product_info = response_content.split(',')
        product_name = product_info[0].split(':')[1].strip()
        quantity = product_info[1].split(':')[1].strip()
        return product_name, quantity
    except Exception as e:
        print(f"Error parsing response: {e}")
        return None, None

def map_name_to_product_id(product_name, products_df):
    # Extract the 'name' column as a list for fuzzy matching
    product_names = products_df['name'].tolist()

    # Check if the product name is already a product_id
    if product_name in products_df['product_id'].values:
        return product_name

    # Find the best match for the product name using fuzzy matching
    best_match, score = process.extractOne(product_name, product_names, scorer=process.fuzz.partial_ratio)

    if score >= 80:  # Set a threshold for the match score
        matched_product_id = products_df.loc[products_df['name'] == best_match, 'product_id'].values[0]
        return matched_product_id
    else:
        print(f"No good match found for product name '{product_name}'. Best match was '{best_match}' with a score of {score}.")
        return None

def extract_and_map_product_info(message, products_df):
    response_content = extract_product_info_from_message(message)
    product_name, quantity = parse_product_info(response_content)
    
    if product_name:
        product_id = map_name_to_product_id(product_name, products_df)
        return product_id, quantity
    else:
        return None, None

def get_stock_for_product(product_id, products_df):
    # Search for the product in the DataFrame using the product_id
    product_row = products_df.loc[products_df['product_id'] == product_id]
    
    # Check if the product exists in the DataFrame
    if not product_row.empty:
        # Extract the stock amount for the found product
        stock = product_row['stock'].values[0]
        return stock
    else:
        print(f"No product found with product_id '{product_id}'.")
        return None

def interpret_quantity(quantity_text, stock):
    # Construct the prompt for OpenAI to interpret the quantity
    prompt = (
        f"Given that the stock available is {stock} and the quantity is {quantity}. Considering the following three situations and decide a best answer. Then, please just output one integer without any explanation: \n\n"
        "If the quantity is a specific number, return that number. "
        "If the quantity is vague, return a reasonable estimate."
        "If the quantity refers to all available stock, return the stock amount. "
    )
    
    # Call the OpenAI API to get the response
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    # Parse the response and interpret the quantity
    response_content = completion.choices[0].message.content.strip()
    print(response_content)
    try:
        interpreted_quantity = int(response_content)  # Convert the interpreted quantity to an integer
    except ValueError:
        print(f"Unable to interpret quantity: {response_content}")
        interpreted_quantity = None
    
    return interpreted_quantity

def determine_order_status(product_id, quantity_text, products_df):
    # Get the stock for the product
    stock = get_stock_for_product(product_id, products_df)
    
    if stock is None:
        return "out of stock", stock, 0 
    
    # Interpret the quantity
    quantity = interpret_quantity(quantity_text, stock)
    
    if quantity is None:
        return "out of stock", stock, 0
    
    # Compare quantity with stock to determine status
    if quantity <= stock:
        return "created", stock - quantity, quantity  # Update stock level
    else:
        return "out of stock", stock, 0


# message = "Hello, I'd want to order one of your Versatile Scarves, the one that can be worn as a scarf, shawl, or headwrap. Thanks!" 
# message = "Hi there, I want to order all the remaining LTH0976 Leather Bifold Wallets you have in stock. I'm opening up a small boutique shop and these would be perfect for my inventory. Thank you!"
# message = "Hi, I'd like to order three to four SFT1098 Infinity Scarves please. My wife loves collecting scarves in different colors and patterns."
message = "Hi, this is Liz. Please send me 5 CLF2109 Cable Knit Beanies and 2 pairs of FZZ1098 Fuzzy Slippers. I'm prepping some holiday gift baskets."
product_id, quantity = extract_and_map_product_info(message, products_df)
stock = get_stock_for_product(product_id, products_df)
print(f"Extracted Product ID: {product_id}, Quantity: {quantity}, stock: {stock}")

status, updated_stock, updated_quantity = determine_order_status(product_id, quantity, products_df)
print(f"status: {status}, quantify(stock): {updated_stock}, updated quantity: {updated_quantity}")

Extracted Product ID: CLF2109, Quantity: 5
Product Name, stock: 2
2
status: created, quantify(stock): 0, updated quantity: 2


In [3]:
classification_df = pd.read_excel('email_results.xlsx', sheet_name='email-classification')
emails_df = pd.read_excel('/Users/jitongd/Desktop/crossover/Solving_Business_Problems_with_AI.xlsx', sheet_name='emails')
products_df = pd.read_excel('/Users/jitongd/Desktop/crossover/Solving_Business_Problems_with_AI.xlsx', sheet_name='products')
order_requests_df = classification_df[classification_df['category'] == 'order request']
order_status_results = []

# Process each order request
for _, row in order_requests_df.iterrows():
    email_id = row['email_id']
    
    # Find the corresponding message in the emails_df
    email_message = emails_df.loc[emails_df['email_id'] == email_id, 'message'].values[0]
    
    # Extract product ID and quantity from the message
    product_id, quantity_text = extract_and_map_product_info(email_message, products_df)
    
    # Determine the order status based on stock availability
    status, updated_stock, updated_quantity = determine_order_status(product_id, quantity_text, products_df)
    
    # Update the stock in the products_df
    if status == "created":
        products_df.loc[products_df['product_id'] == product_id, 'stock'] = updated_stock
    
    # Append the result to the list
    order_status_results.append({
        'email_id': email_id,
        'product_id': product_id,
        'quantity': updated_quantity,
        'status': status
    })


order_status_df = pd.DataFrame(order_status_results)
with pd.ExcelWriter('/Users/jitongd/Desktop/crossover/email_results.xlsx', mode='a', engine='openpyxl') as writer:
    order_status_df.to_excel(writer, sheet_name='order-status', index=False)

4
5
2
5
1
5
Error parsing response: list index out of range
No product found with product_id 'None'.
0
2


RESPOND TO ORDER REQUEST

After the status for each order request has been updated, an email from the store is sent to each customer. Two versons are prepared according to two order status: 'created' and 'out-of-stock'. 

Room for improvement:
1) for created orders with not enough stock, explain that all the available stocks were given to you but still not able to satisfy your total request quantity 

The results are outputed in 'order-response' sheet in table 'email_results.xlsx'. 

In [4]:
import pandas as pd

# Load the order status results
order_status_df = pd.read_excel('/Users/jitongd/Desktop/crossover/email_results.xlsx', sheet_name='order-status')

# Load the original emails data to get customer contact details if needed
products_df = pd.read_excel('/Users/jitongd/Desktop/crossover/Solving_Business_Problems_with_AI.xlsx', sheet_name='products')

# Prepare a list to collect the response emails
response_emails = []

def generate_response_email(email_id, product_id, quantity, status, products_df):
    # Get the product name from the product ID
    # Try to find the product name using the product_id
    try:
        product_name = products_df.loc[products_df['product_id'] == product_id, 'name'].values[0]
    except IndexError:
        # If the product_id is not found, handle the error
        product_name = "Unknown Product"
        print(f"Warning: Product ID '{product_id}' not found in the product catalog.")


    # Generate email content based on the status
    if status == "created":
        response = (
            f"Dear Customer,\n\n"
            f"We are pleased to inform you that your order for {quantity} units of {product_name} (Product ID: {product_id}) "
            "has been successfully processed. Your items will be shipped to you shortly.\n\n"
            "Thank you for shopping with us!\n\n"
            "Best regards,\n"
            "The Fashion Store Team"
        )
    elif status == "out of stock":
        response = (
            f"Dear Customer,\n\n"
            f"We regret to inform you that your order for {quantity} units of {product_name} (Product ID: {product_id}) "
            "could not be fulfilled due to insufficient stock. Unfortunately, this item is currently out of stock.\n\n"
            "We apologize for the inconvenience. You may choose to wait for the item to be restocked, or you can explore alternative products "
            "available on our website. Please let us know how you would like to proceed.\n\n"
            "Thank you for understanding.\n\n"
            "Best regards,\n"
            "The Fashion Store Team"
        )
    return response

# Generate responses for each order
for _, row in order_status_df.iterrows():
    email_id = row['email_id']
    product_id = row['product_id']
    quantity = row['quantity']
    status = row['status']
    
    # Generate the response email content
    response_email = generate_response_email(email_id, product_id, quantity, status, products_df)
    
    # Append the result to the list
    response_emails.append({
        'email_id': email_id,
        'response': response_email
    })


response_emails_df = pd.DataFrame(response_emails)
with pd.ExcelWriter('/Users/jitongd/Desktop/crossover/email_results.xlsx', mode='a', engine='openpyxl') as writer:
    response_emails_df.to_excel(writer, sheet_name='order-response', index=False)


RESPOND TO ORDER INQUIRY

For messageds identifed as order inquiries, emails answering the inquiries are generated. The answers look for the product_id in the messages. 


Room for improvement:
1) Some messages do not have product_id mentioned but have description for the wanted products. Fuzzy matching could be further used. 
2) The content for the answering email could be updated more diversely. 


The results are outputed in 'inquiry_response' sheet in table 'email_results.xlsx'. 

In [5]:
client = OpenAI(
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    api_key='a0BIj000001UF1bMAG'
)

def extract_product_info_from_message(message):
    # Construct the prompt for the model
    prompt = (
        "What does the customer want to buy (if there is a product_id starting with three capital letters, just return the product_id (no other texts)):\n\n"
        f"{message}\n\n"
    )

    # Call the OpenAI API to get the response
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    # Correctly access the message content from the response
    response_content = completion.choices[0].message.content.strip()
    return response_content

def search_product_catalog(product_name, products_df):
    # Assume product_name is a product_id and search for it in the products_df
    matching_products = products_df[products_df['product_id'].str.contains(product_name, case=False, na=False)]
    
    return matching_products


def generate_product_inquiry_response(message, products_df):
    # Extract product information from the customer's message
    product_name = extract_product_info_from_message(message)
    print(product_name)
    
    # Search the catalog for matching products in both name and category
    matching_products = search_product_catalog(product_name, products_df)
    
    if matching_products.empty:
        response = (
            "Dear Customer,\n\n"
            "Thank you for your inquiry. Unfortunately, we could not find any products that match your description. "
            "Please provide more details or check our website for a wider selection.\n\n"
            "Best regards,\nThe Fashion Store Team"
        )
    else:
        product_list = "\n".join([
            f"- {row['name']} (Product ID: {row['product_id']}, Category: {row['category']})\n"
            f"  Description: {row['description']}\n"
            f"  Seasons: {row['seasons']}"
            for _, row in matching_products.iterrows()
        ])
        response = (
            f"Dear Customer,\n\n"
            f"Thank you for your inquiry. Based on your description, we found the following products that might interest you:\n\n"
            f"{product_list}\n\n"
            "We hope you find what you're looking for! If you need further assistance, feel free to reach out.\n\n"
            "Best regards,\nThe Fashion Store Team"
        )
    
    return response


# Example usage:
#message = "Good morning, I'm looking to buy the VBT2345 Vibrant Tote bag. My name is Jessica and I love tote bags, they're so convenient for carrying all my stuff. Last summer I bought this really cute straw tote that I used at the beach. Oh, and a few years ago I got this nylon tote as a free gift with purchase that I still use for groceries."
message = "Hello, I need a new bag to carry my laptop and documents for work. My name is David and I'm having a hard time deciding which would be better - the LTH1098 Leather Backpack or the Leather Tote? Does one have more organizational pockets than the other? Any insight would be appreciated!"
response_email = generate_product_inquiry_response(message, products_df)
print(response_email)


LTH1098
Dear Customer,

Thank you for your inquiry. Based on your description, we found the following products that might interest you:

- Leather Backpack (Product ID: LTH1098, Category: Bags)
  Description: Upgrade your daily carry with our leather backpack. Crafted from premium leather, this stylish backpack features multiple compartments, a padded laptop sleeve, and adjustable straps for a comfortable fit. Perfect for work, travel, or everyday use.
  Seasons: All seasons

We hope you find what you're looking for! If you need further assistance, feel free to reach out.

Best regards,
The Fashion Store Team


In [6]:
import pandas as pd

# Load the email classification and emails data
classification_df = pd.read_excel('email_results.xlsx', sheet_name='email-classification')

# Filter for product inquiry emails
product_inquiry_df = classification_df[classification_df['category'] == 'product inquiry']
emails_df = pd.read_excel('Solving_Business_Problems_with_AI.xlsx', sheet_name='emails')
products_df = pd.read_excel('Solving_Business_Problems_with_AI.xlsx', sheet_name='products')


# Prepare a list to store the responses
response_emails = []

for _, row in product_inquiry_df.iterrows():
    email_id = row['email_id']
    
    # Get the corresponding message from the emails sheet
    email_message = emails_df.loc[emails_df['email_id'] == email_id, 'message'].values[0]
    
    # Generate a response for the product inquiry
    response = generate_product_inquiry_response(email_message, products_df)
    
    # Append the response to the list
    response_emails.append({
        'email_id': email_id,
        'response': response
    })


response_emails_df = pd.DataFrame(response_emails)
with pd.ExcelWriter('email_results.xlsx', mode='a', engine='openpyxl') as writer:
    response_emails_df.to_excel(writer, sheet_name='inquiry-response', index=False)


VBT2345
LTH1098
CSH1098
CBT8901
DHN0987
RSG8901
The customer is interested in buying a work bag, specifically a messenger bag or briefcase style option. The product_id is not provided in the text.
The customer wants to buy slide sandals for men in the Men's Shoes category.
It seems the customer has not specified a product_id in their query.
The customer did not provide a product_id in the request.
The customer wants to buy a Saddle bag.
SDE2345
